# Introduction
Nail Biting is a very serious problem that affects 1 out of 3 individuals based on our estimation. Many of these individuals do it unconsciously, making it a very difficult habit to unlearn. We can make a wearable device that reminds the user when they bite their nails, but we will need a robust and non-intrusive method to detect nail biting. Our project will consist of a device that is worn on the wrist that constantly monitors the user for nail biting. We will use machine learning algorithms and methods to classify the recorded data as either nail biting or not nail biting.

# Data
The dataset is collected from the inertial measurement unit (IMU) of a custom wrist-worn device. One of the team members wore the device while sitting at a desk. The data was then collected across two separate sessions, one with nail-biting and one without. There are 6 input dimensions, namely 3 acceleration dimensions and 3 rotation dimensions. Of course, there is 1 binary output that is either 1 or -1.

# Machine Learning
We decided to use a support vector machine (SVM) for the machine learning of the project. We used the SKLearn impementation of SVM and was able to get very good results.

We also tried using logistic regression from SKLearn, which performed better on paper, but was not chosen for practical considerations that we will discuss later.

# Results
We trained the SVM and logistic regression model on 90% of the data we collected and validated the models using the remaining 10% of the data. We were able to get 0 training misclassifications on both models, while the SVM produced 1 validation misclassification and the logistic regression produced 0 validation misclassifications.

We ultimately decided to use the SVM and not the logistic regression because the SVM performed better in a real-time prediction application. We connected the device to our computer and used both models to perform real-time predictions so that we could test a wider variety of situations. We found that the logistic regression was less stable under erratic movements, such as shaking your fists or rotating your wrists. The output of the logistic regression became unpredictable under these conditions, switching between 1 and -1. The SVM, however, would consistently classify all these situations as not biting hands, which is correct and desirable because nobody bites their nails with such violence. The logistic regression performed better on the dataset, but was less robust in more challenging situations, so we believe that the SVM is the better model for this application.

# Improvements
One issue that affects both the SVM and the logistic regression is false positives. In the dataset, most inputs with the "biting" label consist of an upright arm and thus very similar accelerometer data due to the gravity vector. Therefore, both models would classify any upright arm position as "biting", which is not always correct.

One way to fix this issue is to collect better and more diverse data. Currently, we train the models on a dataset that is only comprised of sitting at a desk and either biting or not biting nails. Thus, the models are not able to take into account nail-biting in different positions, such as laying down face up, laying down on your side, etc.

A wider and more comprehensive sensor suite could also improve our classifications. We tried integrating a microphone into our device, but the computational requirements of the microphone were too high for our embedded microprocessor. Theoretically, a microphone could have allowed the models to "listen" for nail-biting, which could have mitigated our false positive issue. Unfortunately, it proved too challenging to implement.

# Conclusion
We recorded data from a wrist-worn device and used the data to perform binary classfication on whether the user is nail-biting. We decided to use an SVM and got good agreement with the dataset even with cross-validation. The SVM is also robust to erratic movements and will not classify erratic movements as nail-biting.

In [58]:
# import autograd-wrapped numpy
import autograd.numpy as np
from sklearn import svm
from sklearn.linear_model import LogisticRegression

datapath = "Datasets\\"

In [61]:
# load in data
csvname = 'sitting_at_desk_biting.csv'
data_biting = np.loadtxt(datapath + csvname, delimiter = ',')

csvname = 'sitting_at_desk_not_biting.csv'
data_not_biting = np.loadtxt(datapath + csvname, delimiter = ',')

# training dataset
x_train = np.zeros((930 + 2545, 6))
y_train = np.zeros(930 + 2545)
x_train[:930,:] = data_biting[:930]
x_train[930:,:] = data_not_biting[:2545]
y_train[:930] = 1
y_train[930:] = -1

# validation dataset
x_validate = np.zeros((100 + 283, 6))
y_validate = np.zeros(100 + 283)
x_validate[:100,:] = data_biting[930:]
x_validate[100:,:] = data_not_biting[2545:]
y_validate[:100] = 1
y_validate[100:] = -1

# SVM from sklearn
clf = svm.SVC()
clf.fit(x_train, y_train)

# Logistic Regression from sklearn
logistic_model = LogisticRegression().fit(x, y)

# calculate misclassifications
miss_train = np.size(y_train) - np.sum(clf.predict(x_train) == y_train)
miss_validate = np.size(y_validate) - np.sum(clf.predict(x_validate) == y_validate)

print("SVM:")
print("There are", miss_train, "misclassifications from the training dataset")
print("There are", miss_validate, "misclassifications from the validation dataset")

miss_train = np.size(y_train) - np.sum(logistic_model.predict(x_train) == y_train)
miss_validate = np.size(y_validate) - np.sum(logistic_model.predict(x_validate) == y_validate)

print("\nLogistic Regression:")
print("There are", miss_train, "misclassifications from the training dataset")
print("There are", miss_validate, "misclassifications from the validation dataset")

SVM:
There are 1 misclassifications from the training dataset
There are 0 misclassifications from the validation dataset

Logistic Regression:
There are 0 misclassifications from the training dataset
There are 0 misclassifications from the validation dataset
